In [2]:
%pylab inline

from tqdm import tqdm_notebook as tqdm
import requests
import pickle
import re
import itertools
import functools
import collections
import string
import time
from bs4 import BeautifulSoup
from sortedcontainers import SortedList # http://www.grantjenks.com/docs/sortedcontainers/sortedlist.html#sortedlist

from adventlib import *

YEAR = 2021
DAY = int('24')

submit1, submit2 = generate_submits(YEAR, DAY)

while True:
  try:
    raw_input = get_input(YEAR, DAY)
    break
  except Exception as e:
    print(e)
    time.sleep(1)

Populating the interactive namespace from numpy and matplotlib


In [16]:
lines, groups = linesplit("""
inp w
add z w
mod z 2
div w 2
add y w
mod y 2
div w 2
add x w
mod x 2
div w 2
mod w 2
""".strip())

In [40]:
lines, groups = linesplit("""
inp z
inp x
mul z 3
eql z x
""".strip())

In [56]:
lines, groups = linesplit("""
inp w
mul x 0
add x z
mod x 26
div z 1
add x 12
eql x w
eql x 0
mul y 0
add y 25
mul y x
add y 1
mul z y
mul y 0
add y w
add y 7
mul y x
add z y
""".strip())

In [55]:
lines, groups = linesplit(raw_input, verbose=True)

252 lines
1 groups
>>>
inp w
mul x 0
add x z
...


In [57]:
import sympy

In [58]:
def sym_eq(x, y):
  return 1 - sympy.Abs(sympy.sign(x - y))

In [59]:
inputs = sympy.symbols('input:1:14')
input_i = 0
x, y, z, w = 0, 0, 0, 0

for line in lines:
  if m := re.findall(r'^inp (.*)$', line):
    t = inputs[input_i]
    input_i += 1
    var1 = m[0][0]
    if var1 == 'x':
      x = t
    elif var1 == 'y':
      y = t
    elif var1 == 'z':
      z = t
    elif var1 == 'w':
      w = t
  elif m := re.findall(r'^add (.*) (.*)$', line):
    var2 = m[0][1]
    if var2 == 'x':
      t = x
    elif var2 == 'y':
      t = y
    elif var2 == 'z':
      t = z
    elif var2 == 'w':
      t = w
    else:
      t = int(var2)
    
    var1 = m[0][0]
    if var1 == 'x':
      x = x + t
    elif var1 == 'y':
      y = y + t
    elif var1 == 'z':
      z = z + t
    elif var1 == 'w':
      w = w + t
  elif m := re.findall(r'^div (.*) (.*)$', line):
    var2 = m[0][1]
    if var2 == 'x':
      t = x
    elif var2 == 'y':
      t = y
    elif var2 == 'z':
      t = z
    elif var2 == 'w':
      t = w
    else:
      t = int(var2)
    
    var1 = m[0][0]
    if var1 == 'x':
      x = x // t
    elif var1 == 'y':
      y = y // t
    elif var1 == 'z':
      z = z // t
    elif var1 == 'w':
      w = w // t
  elif m := re.findall(r'^mod (.*) (.*)$', line):
    var2 = m[0][1]
    if var2 == 'x':
      t = x
    elif var2 == 'y':
      t = y
    elif var2 == 'z':
      t = z
    elif var2 == 'w':
      t = w
    else:
      t = int(var2)
    
    var1 = m[0][0]
    if var1 == 'x':
      x = x % t
    elif var1 == 'y':
      y = y % t
    elif var1 == 'z':
      z = z % t
    elif var1 == 'w':
      w = w % t
  elif m := re.findall(r'^mul (.*) (.*)$', line):
    var2 = m[0][1]
    if var2 == 'x':
      t = x
    elif var2 == 'y':
      t = y
    elif var2 == 'z':
      t = z
    elif var2 == 'w':
      t = w
    else:
      t = int(var2)
    
    var1 = m[0][0]
    if var1 == 'x':
      x = x * t
    elif var1 == 'y':
      y = y * t
    elif var1 == 'z':
      z = z * t
    elif var1 == 'w':
      w = w * t
  elif m := re.findall(r'^eql (.*) (.*)$', line):
    var2 = m[0][1]
    if var2 == 'x':
      t = x
    elif var2 == 'y':
      t = y
    elif var2 == 'z':
      t = z
    elif var2 == 'w':
      t = w
    else:
      t = int(var2)
    
    var1 = m[0][0]
    if var1 == 'x':
      x = sym_eq(x, t)
    elif var1 == 'y':
      y = sym_eq(y, t)
    elif var1 == 'z':
      z = sym_eq(z, t)
    elif var1 == 'w':
      w = sym_eq(w, t)
  else:
    print('unrec ', line)

In [60]:
x, y, z, w

(1 - Abs(sign(Abs(sign(input00 - 12)) - 1)),
 (1 - Abs(sign(Abs(sign(input00 - 12)) - 1)))*(input00 + 7),
 (1 - Abs(sign(Abs(sign(input00 - 12)) - 1)))*(input00 + 7),
 input00)

In [73]:
sympy.refine?

Signature: sympy.refine(expr, assumptions=True)
Docstring:
Simplify an expression using assumptions.

Explanation

Unlike :func:`~.simplify()` which performs structural simplification
without any assumption, this function transforms the expression into
the form which is only valid under certain assumptions. Note that
``simplify()`` is generally not done in refining process.

Refining boolean expression involves reducing it to ``S.true`` or
``S.false``. Unlike :func:`~.ask()`, the expression will not be reduced
if the truth value cannot be determined.

Examples

>>> from sympy import refine, sqrt, Q
>>> from sympy.abc import x
>>> refine(sqrt(x**2), Q.real(x))
Abs(x)
>>> refine(sqrt(x**2), Q.positive(x))
x

>>> refine(Q.real(x), Q.positive(x))
True
>>> refine(Q.positive(x), Q.real(x))
Q.positive(x)

See Also

sympy.simplify.simplify.simplify : Structural simplification without assumptions.
sympy.assumptions.ask.ask : Query for boolean expressions using assumptions.
File:      ~/projects

In [75]:
x.simplify()

Piecewise((1, ((input00 > 12) & Eq(Abs((input00 - 12)/Abs(input00 - 12)) - 1, 0)) | ((input00 < 12) & Eq(Abs((input00 - 12)/Abs(input00 - 12)) - 1, 0))), (0, Eq(input00, 12)), (1 - Abs((Abs((input00 - 12)/Abs(input00 - 12)) - 1)/Abs(Abs((input00 - 12)/Abs(input00 - 12)) - 1)), True))

In [69]:
sympy.Eq(x, y) + 1

TypeError: unsupported operand type(s) for +: 'Equality' and 'int'

In [46]:
z.subs({
  inputs[0]: 1,
  inputs[1]: 2,
})

0